In [1]:
import os
import json

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object

import csv
from library_dicom.dicom_processor.model.Series import Series

In [5]:
series_paths = get_series_path("/media/salim/DD 2To/FLIP/DICOM")
export_folder = '/media/salim/DD 2To/FLIP/FLIP_JSON'

In [6]:
index_problem = []
for serie_path in series_paths:
    print(series_paths.index(serie_path))
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
    except Exception as err:
        print(err)
        print(dicomsInfo)
        index_problem.append(series_paths.index(serie_path))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
print(len(index_problem))
print(index_problem)

0
[]


In [8]:
json_directory = '/media/salim/DD 2To/FLIP/FLIP_JSON'
list_json = os.listdir(json_directory)
number_of_json = len(list_json)
print(number_of_json)

350


In [ ]:
#POUR PET0 : PT CT ET CSV POUR MASK 

In [3]:
#recuprer liste de liste [[json 1, json 2, uid ]] à partir du csv pour chaque patient

csv_pet0_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/petIni.csv'

with open(csv_pet0_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [4]:
len(csv_data)

63

In [5]:
json_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/FLIP_JSON'
list_json = os.listdir(json_directory)
number_of_json = len(list_json)

In [49]:
all_list = []

for item in csv_data :
    patient_name = item[0]
    patient_id = item[1]
    study_uid = item[2]

    #dans list des json, recupérer ceux dont le Study UID est la même 

    liste = []

    for number_json in range(number_of_json) : 
 
        
        with open(os.path.join(json_directory, list_json[number_json])) as json_file :
            reader = json.load(json_file)
            data = []
        
            for info in reader['study']['StudyInstanceUID'] : #study instance uid of each serie 
                data.append(info)
                study_uid_json = "".join(data)

                if study_uid == study_uid_json : 
                    liste.append(os.path.join(json_directory, list_json[number_json]))
                    
                    
    liste.append(patient_name)
    liste.append(patient_id)
    liste.append(study_uid)
    

    all_list.append(liste)

In [50]:
print(len(all_list))

63


In [4]:
#Clean dataset : if no json, if 2 CT or 2 PT, if same study

None


In [61]:
#Pour certains Study UID,pas de json correspondant, on supprime de la liste  : 
no_json = []
for liste in all_list : 
    if len(liste) != 5 : 
        no_json.append(liste)

print(len(no_json))

for serie in no_json : 
    all_list.remove(serie)

print(len(all_list))

0
63


In [62]:
#doublon
complete_liste = []
complete_liste.append(all_list[0])
for i in range(1, len(all_list)) :
    if all_list[i] not in complete_liste : 
        complete_liste.append(all_list[i])

print(len(complete_liste))

63


In [63]:
#vérifier si les 2 series récupérer sont bien CT et PT , si 2 CT ou 2 PT, supprimer l'item de la liste 
modality = []

for liste in complete_liste : 
    subliste = []

    for i in range(2) : 

        with open(liste[i]) as json_file : 
            data = []
            reader = json.load(json_file)
            for info in reader['series']['Modality'] :
                data.append(info)
                modality1 = "".join(data)
            subliste.append(modality1)
    modality.append(subliste)

#print(modality)
false_modality = []
for modal in modality : 
    index = modality.index(modal)
    if modal[0] == modal[1] :
        false_modality.append(complete_liste[index])

print(len(false_modality))

0


In [64]:
results = {}
study_results = []
list_study_uid = []

for liste in complete_liste:
    subliste = []

    for number_serie in range(2) : 

        with open(liste[number_serie]) as json_file : 
            data = []
            reader = json.load(json_file)
            for info in reader['path']:
                data.append(info)
                path = "".join(data)
            
            
            subliste.append(path)
            subliste.append(modality[complete_liste.index(liste)][number_serie])
        
    subliste.append(liste[-])
    subliste.append(liste[-2])
    study_results.append(subliste) 

print(len(study_results))

63


In [65]:
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'FLIP_list_path', study_results)

In [ ]:
#create json

In [43]:
#create dictionary to save it as json 

study_results = []
list_study_uid = []
list_patient_id = []

for liste in liste_2:
    subliste = []
    list_study_uid.append(liste[-2])
    list_patient_id.append(liste[-1])


    for number_serie in range(2) : 

        with open(liste[number_serie]) as json_file : 
            data = []
            reader = json.load(json_file)
            for info in reader['path']:
                data.append(info)
                path = "".join(data)
            
            
            subliste.append(path)
            subliste.append(modality_2[liste_2.index(liste)][number_serie])
        

    study_results.append(subliste) 

print(len(study_results))


for i in range(len(list_study_uid)):
    for j in range(len(csv_data)):
        if csv_data[j][4] == list_study_uid[i] :

            study_results[i].append(csv_data[j][9])
            study_results[i].append(csv_data[j][1])
            break


print(len(study_results))

579
579


In [44]:
#save json file 
write_json_file('/media/salim/DD 2To/GAINED_NIFTI', 'GAINED_list_path_csv', study_results)

In [23]:
#POUR PET 2 et PET4 : CT ET PET 
csv_pet24_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/data.csv'

with open(csv_pet24_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [25]:
json_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_JSON'
list_json = os.listdir(json_directory)
number_of_json = len(list_json)

In [26]:

all_list = []

for item in csv_data :
    study_uid_csv = item[51]
    patient_id = item[1]

    #dans list des json, recupérer ceux dont le Study UID est la même 

    liste = []

    for number_json in range(number_of_json) : 
 
        
        with open(os.path.join(json_directory, list_json[number_json])) as json_file :
            reader = json.load(json_file)
            data = []
            for info in reader['study']['StudyInstanceUID'] : #study instance uid of each serie 
                data.append(info)
                study_uid_json = "".join(data)

                if study_uid_csv == study_uid_json : 
                    liste.append(os.path.join(json_directory, list_json[number_json]))
    liste.append(patient_id)
    liste.append(study_uid_csv)

    all_list.append(liste)


In [27]:
#Si pas de json, alors on enleve la serie 
print(len(all_list))
no_json = []
for i in range(len(all_list)) :
    if len(all_list[i]) != 4 : 
        no_json.append(all_list[i])

print(len(no_json))

728
134


In [28]:
no_json[50]

['51011103471001', '1.2.826.0.1.3680043.2.455.30.11.5.116865']

In [29]:
for serie in no_json : 
    all_list.remove(serie)

print(len(all_list))

594


In [30]:
#enlever les doublons 
complete_liste = []
complete_liste.append(all_list[0])
for i in range(1, len(all_list)):
    if all_list[i] not in complete_liste :
        complete_liste.append(all_list[i])

print(len(complete_liste))

535


In [31]:
#verifier que CT + PT / si 2 CT ou 2 PT, enlever de la liste 
modality = []

for liste in complete_liste : 
    subliste = []

    for i in range(2) : 

        with open(liste[i]) as json_file : 
            data = []
            reader = json.load(json_file)
            for info in reader['series']['Modality'] :
                data.append(info)
                modality1 = "".join(data)
            subliste.append(modality1)
    modality.append(subliste)

#print(modality)
false_modality = []
for modal in modality : 
    index = modality.index(modal)
    if modal[0] == modal[1] :
        false_modality.append(complete_liste[index])
        false_modality.append(index)

print(len(false_modality))

#remove serie in false_modality from de dataset 

2


In [34]:
error_1 = modality[92]
error_2 = complete_liste[92]

modality.remove(error_1)
complete_liste.remove(error_2)

In [35]:
complete_liste[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_JSON/1.3.46.670589.28.2.10.4.9167.27989.2.3668.0.1364291694.json',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_JSON/1.2.840.113704.1.111.6080.1364289523.8.json',
 '11011101021002',
 '1.2.840.113704.1.111.4636.1364288605.3']

In [36]:
#CREATE JSON WITH PATH CT AND PATH PET
results = {}
study_results = []
list_study_uid = []

for liste in complete_liste:
    subliste = []

    for number_serie in range(2) : 

        with open(liste[number_serie]) as json_file : 
            data = []
            reader = json.load(json_file)
            for info in reader['path']:
                data.append(info)
                path = "".join(data)
            
            
            subliste.append(path)
            subliste.append(modality[complete_liste.index(liste)][number_serie])
        
    subliste.append(liste[-1])
    subliste.append(liste[-2])
    study_results.append(subliste) 

print(len(study_results))

534


In [37]:
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4', 'GAINED_list_path_CT_PET4', study_results)